In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
!pip install gdown

spacy 2.0.16 has requirement regex==2018.01.10, but you'll have regex 2019.3.12 which is incompatible.
You are using pip version 10.0.1, however version 20.2.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

In [4]:
# Model
!gdown --id  1f6WhzP4bdgzy34HBZxlHGmM-Mg1wIGGI

Traceback (most recent call last):
  File "/home/c.marzahl@de.eu.local/anaconda3/lib/python3.6/site-packages/urllib3/contrib/pyopenssl.py", line 441, in wrap_socket
    cnx.do_handshake()
  File "/home/c.marzahl@de.eu.local/anaconda3/lib/python3.6/site-packages/OpenSSL/SSL.py", line 1806, in do_handshake
    self._raise_ssl_error(self._ssl, result)
  File "/home/c.marzahl@de.eu.local/anaconda3/lib/python3.6/site-packages/OpenSSL/SSL.py", line 1546, in _raise_ssl_error
    _raise_current_error()
  File "/home/c.marzahl@de.eu.local/anaconda3/lib/python3.6/site-packages/OpenSSL/_util.py", line 54, in exception_from_error_queue
    raise exception_type(errors)
OpenSSL.SSL.Error: [('SSL routines', 'ssl3_get_server_certificate', 'certificate verify failed')]

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/c.marzahl@de.eu.local/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py", line 595, in urlopen
    self.

In [5]:
import numpy as np
from tqdm import tqdm
from pathlib import Path
import openslide
import pandas as pd
import pickle

In [6]:
from fastai.callbacks.csv_logger import CSVLogger

In [7]:
from object_detection_fastai.helper.object_detection_helper import *
from object_detection_fastai.helper.wsi_loader import *
from object_detection_fastai.loss.RetinaNetFocalLoss import RetinaNetFocalLoss
from object_detection_fastai.models.RetinaNet import RetinaNet
from object_detection_fastai.callbacks.callbacks import BBLossMetrics, BBMetrics, PascalVOCMetric, PascalVOCMetricByDistance

In [8]:
model_file = "RetinaNet-L0-61.p"

model_information = torch.load(model_file, map_location='cpu') \
                    if torch.cuda.is_available() ==  False \
                    else torch.load(model_file)

model_weights = model_information['model']

In [9]:
srep_slides_train = list([
                            "14_EIPH_568381 Turnbull blue.svs", 
                            "26_EIPH_566482 L Berliner Blau.svs",
                            "01_EIPH_563479 Berliner Blau.svs", 
                            "02_EIPH_574162 berliner blau.svs",
                            "28_EIPH_569948 L Turnbull blue.svs", 
                            "08_EIPH_574999 R Turnbull blue.svs", 
                            "03_EIPH_566933 R Berliner Blau neu.svs",    
                            "22_EIPH_575216 Turnbull blue.svs",     
                            "04_EIPH_567017 Berliner Blau.svs",
                            "28_EIPH_569948 L Turnbull blue.svs", 
                            "05_EIPH_569923 Berliner Blau.svs", 
                            "07_EIPH_574882 left lung Turnbull blue.svs", 
                            "20_EIPH_576150 berliner blau.svs", 
                            "08_EIPH_574999 R Turnbull blue.svs", 
                            "01_EIPH_563479 Berliner Blau.svs", 
                            "09_EIPH_563417 L Turnbull blue.svs", 
                            "02_EIPH_574162 berliner blau.svs", 
                            "29_EIPH_566481 L Turnbull blue.svs",
                            "03_EIPH_566933 R Berliner Blau neu.svs",
                            "04_EIPH_567017 Berliner Blau.svs", 
                            "05_EIPH_569923 Berliner Blau.svs",     
                            "27_EIPH_571557 berliner blau.svs",
                            "31_EIPH_568354 Berliner Blau.svs"])

srep_slides_val = list(set(["11_EIPH_575697 Berliner Blau.svs", 
                            "20_EIPH_576150 berliner blau.svs", 
                            "22_EIPH_575216 Turnbull blue.svs"]))

In [10]:
annotations_path = Path("../Statistics/EIPH_Annotations.pkl")
annotations = pd.read_pickle(annotations_path)
annotations_train = annotations[annotations["image_name"].isin(srep_slides_train)]
annotations_val = annotations[annotations["image_name"].isin(srep_slides_val)]
annotations_train.head()

,id,image_id,image_set,species,image_name,image_type,grade,vector,unique_identifier,user_id,deleted,last_editor,data_set_name,version
100398,1421918,3518,245,equine,07_EIPH_574882 left lung Turnbull blue.svs,TurnbullBlue,0,"{'x1': 19619, 'x2': 19747, 'y1': 3009, 'y2': 3...",87e3870c-2316-4689-beaf-d2d6ddf22c28,1,True,1,SREP,SREP
100399,1421919,3518,245,equine,07_EIPH_574882 left lung Turnbull blue.svs,TurnbullBlue,0,"{'x1': 20116, 'x2': 20202, 'y1': 3309, 'y2': 3...",ff289b7a-2fce-4d37-9feb-74b592d842d9,1,True,1,SREP,SREP
100400,1421922,3518,245,equine,07_EIPH_574882 left lung Turnbull blue.svs,TurnbullBlue,0,"{'x1': 14193, 'x2': 14279, 'y1': 3885, 'y2': 3...",6ffcd9c4-1e54-4b9b-b593-4f2c9e85704d,1,True,1,SREP,SREP
100401,1421923,3518,245,equine,07_EIPH_574882 left lung Turnbull blue.svs,TurnbullBlue,0,"{'x1': 14271, 'x2': 14347, 'y1': 3767, 'y2': 3...",1c3cb4f8-fad5-4b13-8859-5b83f31e450a,1,True,1,SREP,SREP
100402,1421924,3518,245,equine,07_EIPH_574882 left lung Turnbull blue.svs,TurnbullBlue,0,"{'x1': 14782, 'x2': 14860, 'y1': 3853, 'y2': 3...",682b6be5-664a-4bd0-ae5a-12a7963a2dd4,1,True,1,SREP,SREP


In [11]:
annotations_val["grade"].unique()

array([0, 1, 2, 3, 4])

In [12]:
slides_path = Path("../../Slides")
files = {slide.name: slide for slide in slides_path.rglob("*.svs")  if slide.name in srep_slides_train + srep_slides_val}
files

{'01_EIPH_563479 Berliner Blau.svs': PosixPath('../../Slides/Equine/01_EIPH_563479 Berliner Blau.svs'),
 '02_EIPH_574162 berliner blau.svs': PosixPath('../../Slides/Equine/02_EIPH_574162 berliner blau.svs'),
 '03_EIPH_566933 R Berliner Blau neu.svs': PosixPath('../../Slides/Equine/03_EIPH_566933 R Berliner Blau neu.svs'),
 '04_EIPH_567017 Berliner Blau.svs': PosixPath('../../Slides/Equine/04_EIPH_567017 Berliner Blau.svs'),
 '05_EIPH_569923 Berliner Blau.svs': PosixPath('../../Slides/Equine/05_EIPH_569923 Berliner Blau.svs'),
 '07_EIPH_574882 left lung Turnbull blue.svs': PosixPath('../../Slides/Equine/07_EIPH_574882 left lung Turnbull blue.svs'),
 '08_EIPH_574999 R Turnbull blue.svs': PosixPath('../../Slides/Equine/08_EIPH_574999 R Turnbull blue.svs'),
 '09_EIPH_563417 L Turnbull blue.svs': PosixPath('../../Slides/Equine/09_EIPH_563417 L Turnbull blue.svs'),
 '11_EIPH_575697 Berliner Blau.svs': PosixPath('../../Slides/Equine/11_EIPH_575697 Berliner Blau.svs'),
 '14_EIPH_568381 Turnbul

In [13]:
tfms = get_transforms(do_flip=True,
                      flip_vert=True,
                      #max_rotate=90,
                      max_lighting=0.0,
                      max_zoom=1.,
                      max_warp=0.0,
                      p_affine=0.5,
                      p_lighting=0.0,
                      #xtra_tfms=xtra_tfms,
                     )
tfms

([RandTransform(tfm=TfmCrop (crop_pad), kwargs={'row_pct': (0, 1), 'col_pct': (0, 1), 'padding_mode': 'reflection'}, p=1.0, resolved={}, do_run=True, is_random=True),
  RandTransform(tfm=TfmAffine (dihedral_affine), kwargs={}, p=1.0, resolved={}, do_run=True, is_random=True),
  RandTransform(tfm=TfmAffine (rotate), kwargs={'degrees': (-10.0, 10.0)}, p=0.5, resolved={}, do_run=True, is_random=True)],
 [RandTransform(tfm=TfmCrop (crop_pad), kwargs={}, p=1.0, resolved={}, do_run=True, is_random=True)])

In [14]:
size = 1024 
level = 0
bs = 16
train_images = 2500
val_images = 1500

In [15]:
def get_y_func(x):
    return x.y

In [16]:
anchors = create_anchors(sizes=[(32,32)], ratios=[1], scales=[0.6, 0.7, 0.9, 1.25, 1.5])

In [ ]:
for i in range(len(srep_slides_train)):
    torch.cuda.empty_cache()

    train_files = []
    val_files = []
    
    experiment_name = "Equine-SREP-V2-FineTune-Ablation_{}".format(i)

    for image_name in srep_slides_train[:i+1]:

        annotations = annotations_train[annotations_train["image_name"] == image_name]
        annotations = annotations[annotations["deleted"] == False]

        slide_path = files[image_name]
        labels =  list(annotations["grade"])
        bboxes = [[vector["x1"], vector["y1"], vector["x2"], vector["y2"]] for vector in annotations["vector"]]
        
        for grade in [0, 1, 2, 3, 4]:
            if grade not in set(labels):
                labels.append(grade)
                bboxes.append([0,0,0,0])

        train_files.append(SlideContainer(slide_path, y=[bboxes, labels],  level=level, width=size, height=size))

    for image_name in annotations_val["image_name"].unique():

        annotations = annotations_val[annotations_val["image_name"] == image_name]
        annotations = annotations[annotations["deleted"] == False]

        slide_path = files[image_name]
        labels =  list(annotations["grade"])
        bboxes = [[vector["x1"], vector["y1"], vector["x2"], vector["y2"]] for vector in annotations["vector"]]

        for grade in [0, 1, 2, 3, 4]:
            if grade not in set(labels):
                labels.append(grade)
                bboxes.append([0,0,0,0])
        
        val_files.append(SlideContainer(slide_path, y=[bboxes, labels],  level=level, width=size, height=size))
    
    train_files = list(np.random.choice(train_files, train_images))
    valid_files = list(np.random.choice(val_files, val_images))
    
    train =  ObjectItemListSlide(train_files, path=slides_path)
    valid = ObjectItemListSlide(valid_files, path=slides_path)
    item_list = ItemLists(slides_path, train, valid)
    lls = item_list.label_from_func(get_y_func, label_cls=SlideObjectCategoryList) #
    lls = lls.transform(tfms, tfm_y=True, size=size)
    data = lls.databunch(bs=bs, collate_fn=bb_pad_collate).normalize()
    
    crit = RetinaNetFocalLoss(anchors)
    encoder = create_body(models.resnet18, True, -2)
    model = RetinaNet(encoder, n_classes=data.train_ds.c, n_anchors=5, sizes=[32], chs=128, final_bias=-4., n_conv=3)
    
    model.load_state_dict(model_weights)
    
    voc = PascalVOCMetricByDistance(anchors, size, [str(i) for i in data.train_ds.y.classes[1:]], radius=25)
    learn = Learner(data, model, loss_func=crit, callback_fns=[BBMetrics, partial(CSVLogger, append=True, filename=experiment_name)], #BBMetrics, ShowGraph
                    metrics=[voc])

    learn.split([model.encoder[6], model.c5top5])
    learn.freeze_to(-2)
    
    learn.fit_one_cycle(3, 1e-3)
    
    learn.unfreeze()
    learn.fit_one_cycle(10, 1e-3)
    #learn.fit_one_cycle(10, 1e-3)
    
    learn.destroy() 

epoch,train_loss,valid_loss,pascal_voc_metric_by_distance,BBloss,focal_loss,AP-0,AP-1,AP-2,AP-3,AP-4,time
0,0.227402,0.304762,0.648172,0.054574,0.250188,0.633187,0.675184,0.610957,0.657185,0.664348,04:11
1,0.205316,0.242091,0.696858,0.049506,0.192584,0.672074,0.739511,0.614498,0.712665,0.745542,04:11
2,0.175733,0.234368,0.705735,0.045992,0.188376,0.648760,0.742472,0.733686,0.651077,0.752679,04:11


epoch,train_loss,valid_loss,pascal_voc_metric_by_distance,BBloss,focal_loss,AP-0,AP-1,AP-2,AP-3,AP-4,time
0,0.170653,0.264366,0.668102,0.048760,0.215607,0.654570,0.749602,0.617160,0.697488,0.621692,04:10
1,0.196473,0.349095,0.551988,0.058972,0.290123,0.418085,0.686307,0.666232,0.667398,0.321917,04:11
2,0.202221,0.295507,0.658232,0.063482,0.232024,0.635163,0.560290,0.686578,0.632100,0.777028,04:15
3,0.197239,0.390021,0.653233,0.076344,0.313677,0.558933,0.607864,0.675612,0.675559,0.748199,04:09
4,0.189104,0.313518,0.636606,0.063239,0.250279,0.591396,0.702179,0.632979,0.646305,0.610172,04:12
5,0.173381,0.303584,0.590837,0.055355,0.248229,0.614794,0.701305,0.648876,0.506049,0.483160,04:14
6,0.149568,0.326641,0.645095,0.053055,0.273586,0.575852,0.707684,0.706855,0.590329,0.644754,04:12
7,0.133625,0.342713,0.635879,0.057541,0.285172,0.574924,0.726539,0.662986,0.576512,0.638434,04:10
8,0.119716,0.342405,0.645501,0.055898,0.286507,0.570679,0.712091,0.674654,0.640541,0.629539,04:12
9,0.114830,0.350770,0.641907,0.056099,0.294671,0.568130,0.698173,0.710013,0.616020,0.617199,04:10


this Learner object self-destroyed - it still exists, but no longer usable


epoch,train_loss,valid_loss,pascal_voc_metric_by_distance,BBloss,focal_loss,AP-0,AP-1,AP-2,AP-3,AP-4,time
0,0.253514,0.246967,0.628922,0.048344,0.198623,0.714979,0.720899,0.642817,0.418917,0.646996,04:19
1,0.282542,0.241371,0.609731,0.046577,0.194794,0.675366,0.744208,0.612972,0.415184,0.600929,04:16
2,0.227796,0.222093,0.650829,0.043929,0.178164,0.702080,0.739372,0.636192,0.472676,0.703823,04:14


epoch,train_loss,valid_loss,pascal_voc_metric_by_distance,BBloss,focal_loss,AP-0,AP-1,AP-2,AP-3,AP-4,time
0,0.222901,0.226248,0.595477,0.044691,0.181557,0.721014,0.743710,0.638660,0.351379,0.522625,04:21
1,0.244373,0.240394,0.534869,0.047793,0.192600,0.694652,0.722549,0.584002,0.281369,0.391772,04:21
2,0.240941,0.245953,0.566028,0.048634,0.197319,0.677683,0.644492,0.668343,0.399872,0.439748,04:20
3,0.245599,0.275322,0.594648,0.059067,0.216255,0.628351,0.652325,0.554352,0.361884,0.776328,04:19
4,0.229764,0.283414,0.488831,0.055478,0.227936,0.635038,0.666989,0.521135,0.222391,0.398602,04:19
5,0.214188,0.242989,0.582770,0.046600,0.196389,0.714279,0.683040,0.622046,0.418694,0.475792,04:20
6,0.213503,0.238989,0.560988,0.044864,0.194125,0.698657,0.740841,0.692468,0.287943,0.385034,04:18
7,0.193197,0.231212,0.550503,0.044992,0.186221,0.687129,0.734030,0.593280,0.356038,0.382041,04:17
8,0.179054,0.231640,0.587967,0.043830,0.187810,0.690716,0.716255,0.694998,0.390340,0.447527,04:18
9,0.176943,0.235774,0.580879,0.042632,0.193142,0.706333,0.707293,0.695834,0.333717,0.461218,04:16


...

epoch,train_loss,valid_loss,pascal_voc_metric_by_distance,BBloss,focal_loss,AP-0,AP-1,AP-2,AP-3,AP-4,time
0,0.252718,0.249539,0.572528,0.053852,0.195688,0.655891,0.751636,0.575955,0.483228,0.395932,04:08
1,0.228166,0.217327,0.670671,0.046582,0.170745,0.709149,0.793534,0.606203,0.486511,0.757957,04:10
2,0.205952,0.203828,0.672745,0.044423,0.159405,0.701187,0.787247,0.667925,0.508366,0.699000,04:08


epoch,train_loss,valid_loss,pascal_voc_metric_by_distance,BBloss,focal_loss,AP-0,AP-1,AP-2,AP-3,AP-4,time
0,0.200703,0.227684,0.636916,0.046824,0.180860,0.703357,0.767995,0.539334,0.449759,0.724134,04:14
1,0.219111,0.251707,0.573077,0.049510,0.202197,0.661619,0.619287,0.703576,0.540465,0.340440,04:09
2,0.224587,0.274576,0.657908,0.062443,0.212133,0.655826,0.704488,0.696784,0.494985,0.737459,04:18
3,0.232094,0.247484,0.645791,0.053421,0.194064,0.652096,0.759364,0.633146,0.396592,0.787758,04:09
4,0.219454,0.231867,0.568138,0.054820,0.177047,0.685706,0.774822,0.671554,0.240095,0.468515,04:12
5,0.209427,0.227953,0.656894,0.049172,0.178781,0.676332,0.733119,0.649740,0.560471,0.664807,04:14
6,0.197094,0.225635,0.601224,0.048772,0.176863,0.740421,0.786163,0.637745,0.377072,0.464716,04:14
7,0.183340,0.224124,0.586088,0.045509,0.178615,0.729784,0.778471,0.605246,0.296184,0.520756,04:15
8,0.172870,0.221042,0.589275,0.045016,0.176026,0.741486,0.758958,0.593929,0.328452,0.523551,04:13
9,0.170356,0.213009,0.610510,0.045209,0.167800,0.749348,0.777538,0.652740,0.351624,0.521299,04:12


this Learner object self-destroyed - it still exists, but no longer usable


epoch,train_loss,valid_loss,pascal_voc_metric_by_distance,BBloss,focal_loss,AP-0,AP-1,AP-2,AP-3,AP-4,time
0,0.241756,0.243346,0.596730,0.049323,0.194024,0.694164,0.701902,0.724351,0.383387,0.479849,04:15
1,0.238255,0.231561,0.641319,0.048405,0.183156,0.687014,0.780654,0.596563,0.570966,0.571397,04:15
2,0.201154,0.207129,0.698336,0.042946,0.164183,0.724426,0.766397,0.645676,0.584132,0.771047,04:09


epoch,train_loss,valid_loss,pascal_voc_metric_by_distance,BBloss,focal_loss,AP-0,AP-1,AP-2,AP-3,AP-4,time
